# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 Matplotlib 样式 (可以在 main_train 外部设置)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 动态导入 main_train ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

# 从我们统一的核心引擎中导入所有需要的函数和模块
from main_train import (
    run_load_config_and_modules,
    run_all_data_pipeline,
    run_preprocess_l3_cache,
    run_hpo_train,
    run_all_models_train,
    run_performance_evaluation,
    run_results_visualization
)

# --- 加载全局配置和模块 ---
# 在 Notebook 的生命周期中，我们只加载一次
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

# **阶段一：数据准备与特征工程**

In [ ]:
USE_LATEST_DATE = True
if config and modules:
    print(f"--- INFO: 日期模式 -> {'使用最新日期' if USE_LATEST_DATE else '使用配置文件中的固定日期'} ---")
    
    # 将开关状态传递给核心引擎函数
    run_all_data_pipeline(config, modules, use_today_as_end_date=USE_LATEST_DATE)

  - INFO: 已将 sh.600019 (sh.600019) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600019_2010-10-27_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600023 (sh.600023) 的日线行情...
  - INFO: 已将 sh.600023 (sh.600023) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600023_2010-10-27_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600025 (sh.600025) 的日线行情...
  - INFO: 已将 sh.600025 (sh.600025) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600025_2010-10-27_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600026 (sh.600026) 的日线行情...
  - INFO: 已将 sh.600026 (sh.600026) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600026_2010-10-27_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600027 (sh.600027) 的日线行情...
  - INFO: 已将 sh.600027 (sh.600027) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600027_2010-10-27_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600028 (sh.600028) 的日线行情...
  - INFO: 已将 sh.600028 (sh.600028) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600028_2010-10-27_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600029 (sh.600029) 的日线行情...
  - INFO: 已将 sh.600029 (sh.600029) 的数据缓存至 data

# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [ ]:
if config and modules:
    # 调用核心引擎中的预处理函数
    # force_reprocess=True/False 可以方便地控制是否重建缓存
    global_data_cache = run_preprocess_l3_cache(config, modules, force_reprocess=False)

### 2.2 超参数优化

In [ ]:
# 同样，可以通过一个简单的开关来控制是否运行
RUN_HPO = False

if RUN_HPO and config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的 HPO 函数
    # HPO 的结果会自动更新到内存中的 config 字典里
    run_hpo_train(config, modules, global_data_cache)

### 2.3 模型训练

In [ ]:
if config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的模型训练函数
    # 同样可以通过开关控制是否强制重训
    all_ic_history = run_all_models_train(
        config, 
        modules,  
        force_retrain_base=False, 
        force_retrain_fuser=True
    )

### 2.4 结果聚合、评估与可视化

In [ ]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)
    else: print('无可视化结果')